In [21]:
import time
import cloudscraper
import requests
import os
import json
from decimal import Decimal
from collections import deque
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from eth_account import Account
from eth_account.messages import encode_typed_data
from datetime import datetime, timedelta, timezone

In [22]:
# 路徑設定
# Chromium 路徑
chrome_path = r"C:\Users\luke\Desktop\chrome-win\chrome.exe"
# ChromeDriver 路徑
chromedriver_path = r"C:\Users\luke\Desktop\chrome-win\chromedriver.exe"
# Chromium User Data 路徑
user_data_dir = r"C:\Users\luke\AppData\Local\Chromium\User Data"

# 個人錢包地址
wallet = "0x0b78d14C3613Fca0569372aB2d43Fc609F6c05D6"

# 1 Ether = 10^18 Wei
WEI_PER_ETHER = Decimal("1000000000000000000")

# 載入配置
with open('config.json') as f:
    config = json.load(f)

# 使用配置中的變數
private_key = config['private_key']
metamask_pasw = config['metamask_pasw']

In [23]:
def setup_driver():
    chrome_options = Options()
    chrome_options.binary_location = chrome_path
    chrome_options.add_argument(f"--user-data-dir={user_data_dir}")
    chrome_options.add_argument("--profile-directory=Default")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--start-maximized")
    prefs = {
    "profile.managed_default_content_settings.images": 2,
    }
    chrome_options.add_experimental_option("prefs", prefs)
    # chrome_options.add_argument("--headless")  

    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def connect_wallet_to_website():
    try:
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//a[text()='Inventory']"))
        )                          
        return True
    except:
        print("目前未連線到網站...開始自動連線")

        try:
            connect_button = WebDriverWait(driver, 6).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text()='Connect']"))
            )
            print("找到連線按鈕")
            connect_button.click()

            connect_metamask_btn = WebDriverWait(driver, 6).until(
                EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[2]/div/div[2]/div/div/button"))
            )
            connect_metamask_btn.click()

            while True:
                if(len(driver.window_handles) > 2):
                    break

            driver.switch_to.window(driver.window_handles[-1])
            driver.maximize_window()
        except:
            return True
        
        # 找密碼輸入框
        try:
            pas_input = WebDriverWait(driver, 6).until(
                    EC.element_to_be_clickable((By.ID, 'password'))
                )
            pas_input.send_keys(metamask_pasw) 
            pas_input.send_keys(Keys.ENTER)
            print("密碼輸入完成")
        except:
            print("未找到密碼輸入框")

            # 找確認按鈕
            try:
                connect_button = WebDriverWait(driver, 6).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[text()='確認']"))
                )
                connect_button.click()
                print("找到確認按鈕")
            except:
                print("未找到確認按鈕")


        try:
            while True:
                try:
                    WebDriverWait(driver, 0.01).until(
                        EC.element_to_be_clickable((By.XPATH, "//h2[text()='Signature request']"))
                    )
                    break
                except:
                    driver.switch_to.window(driver.window_handles[-1])
                
            driver.maximize_window()
            confirm_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[text()='確認']"))
                )
            confirm_button.click()
            print("連線網站成功")
            driver.switch_to.window(driver.window_handles[0])
            return True
        except Exception as e:
            print(e)
            print("連線網站失敗")
            return False

def get_cookies_from_selenium(driver):
    """
    獲取 Selenium 中的 Cookie，並轉換為 requests 可用的格式
    """
    cookies = driver.get_cookies()
    cookie_dict = {cookie['name']: cookie['value'] for cookie in cookies}
    return cookie_dict

def send_post_with_cookies(url, post_data, cookies):
    """
    使用 requests 發送帶有 Cookie 的 POST 請求
    """
    scraper = cloudscraper.create_scraper()
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    }

    response = scraper.post(url, json=post_data, cookies=cookies, headers=headers)
    return response

def fetch_url_using_cloudscraper(method: str, url: str, payload=None):
    scraper = cloudscraper.create_scraper()
    try:
        if method == "post":
            response = scraper.post(url, json=payload)
        else:
            response = scraper.get(url)
        response.raise_for_status()  # 如果不是 200，會觸發 HTTPError
        return response.json()  # 成功時回傳資料
    except cloudscraper.exceptions.CloudflareChallengeError:
        print("Cloudflare challenge encountered. Retrying...")
        return None
    except requests.exceptions.HTTPError as e:
        if response.status_code == 403:
            print("403 Forbidden. Retry after 30 seconds...")
            time.sleep(30)
        elif response.status_code == 429:
            print("Too many requests. Retry after 120 seconds...")
            time.sleep(120)
        else:
            print(f"HTTP error occurred: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


In [24]:
def get_transaction_result(transactionId, cookies):
    transactionId.replace(":", "%3A")
    url = f"https://msu.io/marketplace/api/marketplace/transaction/{transactionId}/result"

    scraper = cloudscraper.create_scraper()
    response = scraper.get(url, cookies=cookies)

    try:
        response.raise_for_status()
        # 將返回結果解析為 JSON
        result = response.json()
        
        return result["code"]
    except json.JSONDecodeError as e:
        print("獲取交易結果失敗")
        print("Error decoding JSON:", e)
    except Exception as e:
        print("獲取交易結果失敗")
        print(f"HTTP Error {response.status_code}: {response.text}")

def fetch_all_pets():
    url = "https://msu.io/marketplace/api/marketplace/explore/items"
    fetch_amount = 7
    payload = {
        "filter": {
            "categoryNo": 1000401001,
            "price": {"min": 0, "max": 10000000000},
        },
        "sorting": "ExploreSorting_RECENTLY_LISTED",
        "paginationParam": {"pageNo": 1, "pageSize": fetch_amount},
    }

    return fetch_url_using_cloudscraper("post", url, payload)


def query_equipment(name = None):
    url = "https://msu.io/marketplace/api/marketplace/explore/items"
    fetch_amount = 5
    payload = {
        "filter": {
            "name": name,
            "categoryNo": 0,
            "price": {"min": 0, "max": 10000000000},
            "level": {"min": 0, "max": 250},
            "starforce": {"min": 0, "max": 25},
            "potential": {"min": 0, "max": 4},
            "bonusPotential": {"min": 0, "max": 4},
        },
        "sorting": "ExploreSorting_LOWEST_PRICE",
        "paginationParam": {"pageNo": 1, "pageSize": fetch_amount},
    }

    return fetch_url_using_cloudscraper("post", url, payload)

def get_singal_pet_skill_info(tokenId: int):
    url = f"https://msu.io/marketplace/api/marketplace/items/{tokenId}"

    response = fetch_url_using_cloudscraper("get", url)
    if response is None:
        return None
    else:
        pet_skills = response["item"]["pet"]["petSkills"]
        return pet_skills

In [30]:
def buy_item_api(tokenId, tokenAmount):
    # 設定 NFT 和 Token 數據

    # 計算時間戳
    current_time = datetime.now(timezone.utc)
    listing_time = int(current_time.timestamp())  # 秒級時間戳
    expiration_time = int((current_time + timedelta(days=1)).timestamp())  # 一天後
    salt = int(current_time.timestamp() * 1000)  # 毫秒級時間戳
    tokenAddress = "0x07E49Ad54FcD23F6e7B911C2068F0148d1827c08"
    nftAddress = "0x43DCff2A0cedcd5e10e6f1c18b503498dDCe60d5"

    # 構建 EIP-712 結構化數據
    full_message = {
        "domain": {
            "name": "Marketplace",
            "version": "1.0",
            "chainId": 68414,
            "verifyingContract": "0xf1c82c082af3de3614771105f01dc419c3163352",
        },
        "primaryType": "Order",
        "types": {
            "EIP712Domain": [
                {"name": "name", "type": "string"},
                {"name": "version", "type": "string"},
                {"name": "chainId", "type": "uint256"},
                {"name": "verifyingContract", "type": "address"},
            ],
            "Order": [
                {"name": "isSeller", "type": "uint256"},  # 維持原狀
                {"name": "maker", "type": "address"},
                {"name": "listingTime", "type": "uint256"},
                {"name": "expirationTime", "type": "uint256"},
                {"name": "tokenAddress", "type": "address"},
                {"name": "tokenAmount", "type": "uint256"},
                {"name": "nftAddress", "type": "address"},
                {"name": "nftTokenId", "type": "uint256"},
                {"name": "salt", "type": "uint256"},
            ],
        },
        "message": {
            "isSeller": 0,  # 簽名時使用數值 0
            "maker": wallet.lower(),
            "listingTime": listing_time,
            "expirationTime": expiration_time,
            "tokenAddress": tokenAddress.lower(),
            "tokenAmount": int(tokenAmount),
            "nftAddress": nftAddress.lower(),
            "nftTokenId": int(tokenId),  # 轉換為整數
            "salt": salt,
        },
    }

    # 簽名
    encoded_message = encode_typed_data(full_message=full_message)
    signature = Account.sign_message(encoded_message, private_key)

    # 組裝 POST 資料
    post_data = {
        "order": {
            "isSeller": False,  # 修改為bool False
            "maker": wallet,
            "listingTime": str(full_message["message"]["listingTime"]),
            "expirationTime": str(full_message["message"]["expirationTime"]),
            "tokenAddress": tokenAddress,
            "tokenAmount": str(full_message["message"]["tokenAmount"]),
            "nftAddress": nftAddress,
            "nftTokenId": str(full_message["message"]["nftTokenId"]),
            "salt": str(full_message["message"]["salt"]), 
        },
        "orderSign": "0x" + signature.signature.hex(),
    }

    url = f"https://msu.io/marketplace/api/marketplace/items/{tokenId}/buy" 

    cookies = get_cookies_from_selenium(driver)

    # 發送 POST 請求
    response = send_post_with_cookies(url, post_data, cookies)

    # 檢查 HTTP 狀態碼
    try:
        response.raise_for_status()
        # 將返回結果解析為 JSON
        result = response.json()
        transactionId = result["transactionId"]
        print(transactionId)

        transaction_result_code = get_transaction_result(transactionId, cookies)
        for _ in range(6):
            if transaction_result_code == 2:
                return True
            else:
                time.sleep(1.5)

        return False

    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return False
    except Exception as e:
        print(f"HTTP Error {response.status_code}: {response.text}")
        return False

def auto_buy_pet():
    # 上次抓取到的寵物ID
    last_pet_token_id = None

    while True:
        all_pets_list = fetch_all_pets()
        # 沒有抓到值時跳過當次迴圈
        if all_pets_list is None:
            continue
        else:
            all_pets_list = all_pets_list["items"]
            first_pet_id = all_pets_list[0]["tokenId"]

        for i in all_pets_list:
            tokenId = i["tokenId"]
            # 遇到上一次抓取過的寵物就停止
            if last_pet_token_id == tokenId:
                break

            skill_info = get_singal_pet_skill_info(tokenId)
            if skill_info is None:
                # 沒有抓到值時跳過當次迴圈
                continue

            pet_skills = set(skill_info)
            print(pet_skills)
            # 篩選條件: [{技能}, 可接受最高價格]
            filters = [
                [set({}), 1000], #完全不符合
                [{"Expanded Auto Move", "Auto Move"}, 10000],
                [{"Auto Buff"}, 385501],
                [{"Magnet Effect"}, 385501],
            ]

            # 檢查每組篩選條件是否符合
            for filter_set in filters:
                # 檢查 filter_set 是否在 pet_skills 中
                if filter_set[0].issubset(pet_skills):
                    price = Decimal(i["salesInfo"]["priceWei"]) / WEI_PER_ETHER
                    print(price)
                    rounded_price = round(price, 1)
                    if rounded_price <= filter_set[1]:
                        formatted_skills = "\n".join(filter_set[0])
                        message = f"\n發現高價值寵物\n價格: ${price}\n技能:\n{formatted_skills}\nhttps://msu.io/marketplace/nft/{tokenId}"
                        print(message)
                
                        result = buy_item_api(tokenId, i["salesInfo"]["priceWei"])
                        if result:
                            print("已購買成功")
                            
                        driver.get("https://msu.io/marketplace/inventory/0x0b78d14C3613Fca0569372aB2d43Fc609F6c05D6")
                        break

            time.sleep(0.1)

        last_pet_token_id = first_pet_id

        time.sleep(8)
        driver.refresh()
        connect_wallet_to_website()


def auto_buy_equipment():
    while True:
        euipments_list = query_equipment("Golden Clover Belt")
        # 沒有抓到值時跳過當次迴圈
        if euipments_list is None:
            continue
        else:
            euipments_list = euipments_list["items"]

        for i in euipments_list:
            # 設定可接受的價格上限
            price_limit = 770

            tokenId = i["tokenId"]
            price = int(int(i["salesInfo"]["priceWei"]) / WEI_PER_ETHER)
            print(price)

            if price <= price_limit:
                message = f"\n發現高價值裝備\n價格: ${price}\nhttps://msu.io/marketplace/nft/{tokenId}"
                print(message)

                result = buy_item_api(tokenId, i["salesInfo"]["priceWei"])
                
                if result:
                    print("已購買成功")
                    
                driver.get("https://msu.io/marketplace/inventory/0x0b78d14C3613Fca0569372aB2d43Fc609F6c05D6")
                break

        time.sleep(8)
        driver.refresh()
        connect_wallet_to_website()


In [26]:
def main():
    global driver
    driver = setup_driver()
    driver.switch_to.window(driver.window_handles[0])
    driver.get("https://msu.io/marketplace/inventory/0x0b78d14C3613Fca0569372aB2d43Fc609F6c05D6?tab=onsale")

    if connect_wallet_to_website():
        print("初始化完成")
    else:
        print("初始化失敗")

main()

目前未連線到網站...開始自動連線
找到連線按鈕
密碼輸入完成
連線網站成功
初始化完成


In [31]:
auto_buy_pet()

{'Auto MP Potion Pouch', 'Fatten Up', 'Auto HP Potion Pouch', 'Item Pouch', 'Auto Move', 'NESO Magnet', 'Expanded Auto Move'}
33333333
33333333
{'Auto MP Potion Pouch', 'Fatten Up', 'Auto HP Potion Pouch', 'Item Pouch', 'NESO Magnet'}
100000
{'Auto MP Potion Pouch', 'Auto HP Potion Pouch', 'Item Pouch', 'NESO Magnet', 'Expanded Auto Move'}
400000
{'Auto MP Potion Pouch', 'Auto HP Potion Pouch', 'Item Pouch', 'NESO Magnet', 'Expanded Auto Move'}
44444444
{'Auto Buff', 'Auto MP Potion Pouch', 'Auto HP Potion Pouch', 'Item Pouch', 'NESO Magnet'}
1369999
1369999
{'Pet Training Skill', 'Auto MP Potion Pouch', 'Fatten Up', 'Auto HP Potion Pouch', 'Item Pouch', 'Auto Move', 'NESO Magnet'}
1698888
{'Auto Buff', 'Auto MP Potion Pouch', 'Auto HP Potion Pouch', 'Item Pouch', 'NESO Magnet'}
1369999
1369999
{'Auto MP Potion Pouch', 'Auto HP Potion Pouch', 'Item Pouch', 'NESO Magnet', 'Expanded Auto Move'}
1000000
{'Pet Training Skill', 'Auto MP Potion Pouch', 'Auto HP Potion Pouch', 'Item Pouch', '

KeyboardInterrupt: 